In [ ]:
# # ----------------- Download Fyers Master List ----------------- #
# import requests
# import pandas as pd
# import os

# # Download the CSV file
# url = "https://public.fyers.in/sym_details/NSE_CM.csv"
# response = requests.get(url)

# # Create directory if it doesn't exist
# data_dir = "D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_Fyers_via_API"
# os.makedirs(data_dir, exist_ok=True)

# # Save the file
# file_path = os.path.join(data_dir, "fyers_master_list.csv")
# with open(file_path, 'wb') as f:
#     f.write(response.content)

# print(f"Master list downloaded and saved to: {file_path}")

# # Load and display basic info
# master_df = pd.read_csv(file_path)


In [ ]:
import pandas as pd
import json
from fyers_apiv3 import fyersModel
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os

CONFIG_FILE = "fyers_config.json"

def get_fyers():
    """Get Fyers client with auto token management"""
    config = json.load(open(CONFIG_FILE))
    
    # Try existing tokens
    if 'access_token' in config:
        fyers = fyersModel.FyersModel(client_id=config['client_id'], token=config['access_token'])
        try:
            if fyers.get_profile().get('s') == 'ok':
                return fyers
        except:
            pass
        
        # Try refresh
        if 'refresh_token' in config:
            try:
                session = fyersModel.SessionModel(
                    client_id=config['client_id'],
                    secret_key=config['secret_key'],
                    redirect_uri=config['redirect_uri'],
                    response_type="code",
                    grant_type="refresh_token"
                )
                session.set_token(config['refresh_token'])
                resp = session.generate_token()
                if resp.get('s') == 'ok':
                    config['access_token'] = resp['access_token']
                    config['refresh_token'] = resp['refresh_token']
                    json.dump(config, open(CONFIG_FILE, 'w'), indent=4)
                    return fyersModel.FyersModel(client_id=config['client_id'], token=resp['access_token'])
            except:
                pass
    
    # New auth
    session = fyersModel.SessionModel(
        client_id=config['client_id'],
        secret_key=config['secret_key'],
        redirect_uri=config['redirect_uri'],
        response_type="code"
    )
    print(f"Visit: {session.generate_authcode()}")
    auth_code = input("Enter auth_code: ").strip()
    
    session.grant_type = "authorization_code"
    session.set_token(auth_code)
    resp = session.generate_token()
    
    config['access_token'] = resp['access_token']
    config['refresh_token'] = resp['refresh_token']
    json.dump(config, open(CONFIG_FILE, 'w'), indent=4)
    return fyersModel.FyersModel(client_id=config['client_id'], token=resp['access_token'])

fyers = get_fyers()

'https://api-t1.fyers.in/api/v3/generate-authcode?client_id=UZJ1B1WF4N-100&redirect_uri=https%3A%2F%2Fwww.google.com%2F&response_type=code&state=None'

In [ ]:
# nifty_50_componants_data_from_tradingview.csv
nifty_comprehensive_df = pd.read_csv("D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/nifty_50_componants_data_from_tradingview.csv")

In [ ]:
# Read Master_list file of Fyers
df = pd.read_csv(r"D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_Fyers_via_API/fyers_master_list.csv")
df = df[df['NSE:GOLDSTAR-SM'].str.contains('-EQ', na=False)].reset_index(drop=True)
df

In [ ]:
# # ----------------------------------- find stock in masterlist file -----------------------------------#

n = 0

# Nifty 50 symbols to find
nifty_50 = nifty_comprehensive_df['Symbol'].values.tolist()
nifty_50 = [symbol.replace("_", "-") for symbol in nifty_50]

x = 0
# Fix: Iterate over the actual values in the list, not indices
for stock in nifty_50:
    if f"NSE:{stock}-EQ" in df["NSE:GOLDSTAR-SM"].values:
        x += 1
        n += 1
        # a = print(x, stock, "\tFound") if len(stock) > 4 else print(x, stock, "\t\tFound")
    else:
        x += 1
        print(x, stock, "\tNot found")

print(f"Total {n} Nifty 50 stocks found in the DataFrame.")

In [ ]:
nifty_50_tokens = [f"NSE:{stock}-EQ" for stock in nifty_50]
filtered_masterlist = df[df['NSE:GOLDSTAR-SM'].isin(nifty_50_tokens)].reset_index(drop=True)
nifty50_stockcode = filtered_masterlist['NSE:GOLDSTAR-SM'].to_list()
print(nifty50_stockcode)

In [ ]:
# Download Nifty 50 components data and save to folder
import os
import time

# Create directory for saving data
save_dir = "D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_Fyers_via_API/nifty50_daily_data"
os.makedirs(save_dir, exist_ok=True)

# Date range for download
date_range = {
    "resolution": "D",
    "date_format": "1", 
    "range_from": "2025-05-01", 
    "range_to": "2025-09-23", 
    "cont_flag": "1"
}

print(f"Starting download of {len(nifty50_stockcode)} Nifty 50 stocks...")
print(f"Date range: {date_range['range_from']} to {date_range['range_to']}")
print(f"Saving to: {save_dir}")

successful_downloads = 0
failed_downloads = []

for i, symbol in enumerate(nifty50_stockcode, 1):
    try:
        # Prepare data for API call
        data = {
            "symbol": symbol,
            **date_range
        }
        
        # Make API call
        response = fyers.history(data=data)
        
        if response.get('s') == 'ok' and response.get('candles'):
            # Create DataFrame
            df = pd.DataFrame(response['candles'], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['datetime'] = pd.to_datetime(df['timestamp'], unit='s', utc=True).dt.tz_convert('Asia/Kolkata')
            df['symbol'] = symbol.replace("NSE:", "").replace("-EQ", "")
            df = df[['symbol', 'datetime', 'open', 'high', 'low', 'close', 'volume']]
            df['datetime'] = df['datetime'].dt.tz_localize(None)
            
            # Save to CSV
            filename = f"{symbol.replace('NSE:', '').replace('-EQ', '')}_daily_data.csv"
            filepath = os.path.join(save_dir, filename)
            df.to_csv(filepath, index=False)
            
            successful_downloads += 1
            print(f"{i:2d}. {symbol:20s} - Downloaded {len(df)} records -> {filename}")
            
        else:
            failed_downloads.append(symbol)
            print(f"{i:2d}. {symbol:20s} - Failed: {response.get('message', 'Unknown error')}")
        
        # Add small delay to avoid rate limiting
        time.sleep(0.1)
        
    except Exception as e:
        failed_downloads.append(symbol)
        print(f"{i:2d}. {symbol:20s} - Error: {str(e)}")

print(f"\nDownload Summary:")
print(f"Successful: {successful_downloads}")
print(f"Failed: {len(failed_downloads)}")
if failed_downloads:
    print(f"Failed symbols: {failed_downloads}")
print(f"Data saved in: {save_dir}")

In [ ]:
# Sum all stocks volume by datetime
import glob

# Path to the folder containing individual stock CSV files
data_folder = "D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_Fyers_via_API/nifty50_daily_data"

# Get all CSV files in the folder
csv_files = glob.glob(os.path.join(data_folder, "*_daily_data.csv"))

print(f"Found {len(csv_files)} CSV files to process...")

# Initialize list to store all dataframes
all_dataframes = []

# Read each CSV file and collect dataframes
for file_path in csv_files:
    try:
        df = pd.read_csv(file_path)
        df['datetime'] = pd.to_datetime(df['datetime'])
        # Keep only datetime and volume columns with symbol info
        df_volume = df[['datetime', 'volume', 'symbol']].copy()
        all_dataframes.append(df_volume)
        # print(f"Loaded: {os.path.basename(file_path)} - {len(df)} records")
    except Exception as e:
        print(f"Error loading {os.path.basename(file_path)}: {str(e)}")

if all_dataframes:
    # Combine all dataframes
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    
    # Group by datetime and sum volumes
    volume_summary = combined_df.groupby('datetime').agg({
        'volume': 'sum',
        'symbol': 'count'  # Count how many stocks contributed to each date
    }).reset_index()
    
    # Rename columns for clarity
    volume_summary.rename(columns={'symbol': 'stock_count'}, inplace=True)
    
    # Sort by datetime
    volume_summary = volume_summary.sort_values('datetime').reset_index(drop=True)
    
    print(f"\nVolume Summary (Total across all Nifty 50 stocks):")
    print("=" * 60)
    print(volume_summary.to_string(index=False))
    
else:
    print("No data files found to process!")

In [ ]:
volume_summary['volume'] = round((volume_summary['volume'] / 1000000), 2)
volume_summary

----------------------- Testing cell ------------------------

In [ ]:
# Create tvml_df with both TradingView and Fyers symbol formats
# Read TradingView Nifty 50 components file
tv_df = pd.read_csv('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/nifty_50_componants_data_from_tradingview.csv')

# Extract symbols and company names
symbols = tv_df['Symbol'].tolist()
company_names = tv_df['Company_Name'].tolist()

# Handle special case for BAJAJ_AUTO
symbols = [symbol.replace("_", "-") for symbol in symbols]

# Create tvml_df with both formats
tvml_df = pd.DataFrame({
    'tv_symbol': symbols,
    'company_names': company_names
})

tvml_df.head()

In [ ]:
# Create n5v_df with symbol and volume for date 2025-08-11
import pandas as pd
import glob
import os

# Path to the folder containing individual stock CSV files
data_folder = "D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_Fyers_via_API/nifty50_daily_data"

# Get all CSV files in the folder
csv_files = glob.glob(os.path.join(data_folder, "*_daily_data.csv"))

# Target date
target_date = '2025-08-22'

# Initialize list to store data for target date
data_for_date = []

# Read each CSV file and extract data for target date
for file_path in csv_files:
    try:
        df = pd.read_csv(file_path)
        df['datetime'] = pd.to_datetime(df['datetime'])
        
        # Filter for target date
        target_data = df[df['datetime'].dt.date == pd.to_datetime(target_date).date()]
        
        if not target_data.empty:
            symbol = target_data['symbol'].iloc[0]
            volume = target_data['volume'].iloc[0]
            data_for_date.append({'symbol': symbol, 'volume': volume})
            
    except Exception as e:
        print(f"Error processing {os.path.basename(file_path)}: {str(e)}")

# Create n5v_df
n5v_df = pd.DataFrame(data_for_date)

# Sort by symbol for better readability
n5v_df = n5v_df.sort_values('symbol').reset_index(drop=True)

print(f"n5v_df - Nifty 50 Volume Data for {target_date}")
print("=" * 50)
print(n5v_df.to_string(index=False))
print(f"\nTotal stocks: {len(n5v_df)}")
print(f"Total volume: {n5v_df['volume'].sum()/1000000:,}")

ea = n5v_df

In [ ]:
# First verify symbol matching between ea and tvml_df
print("Symbol Matching Verification:")
print("=" * 40)

# Get unique symbols from both DataFrames
ea_symbols = set(ea['symbol'].tolist())
tvml_symbols = set(tvml_df['tv_symbol'].tolist())

# Find matches and mismatches
matched_symbols = ea_symbols.intersection(tvml_symbols)
ea_only = ea_symbols - tvml_symbols
tvml_only = tvml_symbols - ea_symbols

print(f"EA symbols: {len(ea_symbols)}")
print(f"TVML symbols: {len(tvml_symbols)}")
print(f"Matched symbols: {len(matched_symbols)}")
print(f"EA only symbols: {len(ea_only)}")
print(f"TVML only symbols: {len(tvml_only)}")

if ea_only:
    print(f"\nSymbols in EA but not in TVML: {sorted(ea_only)}")
if tvml_only:
    print(f"\nSymbols in TVML but not in EA: {sorted(tvml_only)}")

print(f"\nMatched symbols: {sorted(matched_symbols)}")

# After verification, add company names to ea DataFrame
# Note: Using 'company_names' (plural) as per tvml_df column name
ea_with_names = ea.merge(tvml_df[['tv_symbol', 'company_names']], 
                        left_on='symbol', 
                        right_on='tv_symbol', 
                        how='left')

# Drop the duplicate tv_symbol column and reorder columns
ea_with_names = ea_with_names[['symbol', 'company_names', 'volume']].copy()

# Rename column to singular for consistency
ea_with_names.rename(columns={'company_names': 'company_name'}, inplace=True)

print(f"\nEA DataFrame with Company Names:")
print("=" * 50)
ea_with_names.head()

In [ ]:
# save ea_with_names in D:\Programming\Download_Backtest_Deploy\1__Download\1__Download_data_Fyers_via_API
ea_with_names.to_csv('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_Fyers_via_API/ea_with_names.csv', index=False)

___________________ Analysis code ___________________

In [ ]:
import pandas as pd
import numpy as np

# Load both ea_with_names DataFrames
fyers_df = pd.read_csv('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_Fyers_via_API/ea_with_names.csv')
icici_df = pd.read_csv('D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/ea_with_names.csv')

# Merge on company_name to align the data
merged_df = pd.merge(fyers_df, icici_df, on='company_name', suffixes=('_fyers', '_icici'), how='outer')

# Calculate volume differences
merged_df['volume_diff'] = merged_df['volume_fyers'] - merged_df['volume_icici']
merged_df['volume_diff_pct'] = (merged_df['volume_diff'] / merged_df['volume_icici']) * 100

# Total volumes
total_fyers = merged_df['volume_fyers'].sum()
total_icici = merged_df['volume_icici'].sum()
total_diff = total_fyers - total_icici
total_diff_pct = (total_diff / total_icici) * 100
merged_df['pct'] = round((((merged_df['volume_fyers'] - merged_df['volume_icici'])) / merged_df['volume_icici']) * 100, 2)


In [ ]:
merged_df.columns

In [ ]:
merged_df['volume_fyers'] = round((merged_df['volume_fyers'] / 1000000), 2)
merged_df['volume_icici'] = round((merged_df['volume_icici'] / 1000000), 2)

In [ ]:
merged_df